# Lossless Pipeline Demo

This notebook install and runs a minimal version of the Lossless pipeline. It assumes you have successfully run the `test_environment` notebook.

The first step will include cloning down the pipeline into the support directory.

In [ ]:
!git clone https://github.com/Andesha/pylossless.git support/pylossless

We can validate that the cloning was successful by examining the folder we have cloned to:

In [ ]:
!ls support/pylossless

Assuming this looks good, we can point `pip` to this folder.

**NOTE**: We point pip to a path meaning it builds from the version we cloned down. Not a release version that is not actively maintained.

In [ ]:
!pip install support/pylossless

If importing works, then the pipeline is likely to run smoothly.

In [ ]:
import mne
import pylossless as ll

The following creates a default configuration which controls the pipeline

In [ ]:
config = ll.config.Config()
config.load_default()
config.save("ll_demo_config.yaml")
config

Load your own file or use the demo.

**NOTE**: The montage for this file must be correctly set.

In [ ]:
fname = mne.datasets.sample.data_path() / 'MEG' / 'sample' /  'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(fname, preload=True)
raw = raw.pick('eeg')
raw

Run the pipeline based on the created/edited config file:

In [ ]:
pipeline = ll.LosslessPipeline('ll_demo_config.yaml')
pipeline.run_with_raw(raw)

If you would like to see a nice summary of what the pipeline found:

In [ ]:
pipeline

If you would like to apply the changes the pipeline suggested:

**NOTE**: This brute force trusts iclabel which may be problematic for studies. For now it is this way just to demo.

In [ ]:
cleaned_state = pipeline.raw.copy()
cleaned_state.load_data()

pipeline.ica2.exclude = [index for index,comp in pipeline.flags['ic'].iterrows() if comp['ic_type'] in ['eog', 'ecg', 'muscle', 'line_noise', 'channel_noise']]
pipeline.ica2.apply(cleaned_state)

cleaned_state = cleaned_state.interpolate_bads()

cleaned_state